# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [17]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [18]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLut4Simple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut4_simple'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, net_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 8
mini_batch_size       = 64*2
frame_modulation_size = 31

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [19]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [20]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([64*4*4], N=4),
            bb.DifferentiableLut([64*4], N=4, connection='serial'),
            bb.DifferentiableLut([64], N=4, connection='serial'),
            bb.DifferentiableLut([10*4*4], N=4),
            bb.DifferentiableLut([10*4], N=4, connection='serial'),
            bb.DifferentiableLut([10], N=4, connection='serial'),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [21]:
bb.load_networks(data_path, net)

print(net.get_info())

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net)

----------------------------------------------------------------------
[Sequential] 
 input  shape : [1, 28, 28] output shape : [10]
  --------------------------------------------------------------------
  [RealToBinary] 
   input  shape : {1, 28, 28} output shape : {1, 28, 28}
  --------------------------------------------------------------------
  [DifferentiableLut4] 
   input  shape : {1, 28, 28} output shape : {1024}
   connection : random
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut4] 
   input  shape : {1024} output shape : {256}
   connection : serial
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut4] 
   input  shape : {256} output shape : {64}
   connection : serial
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut4] 
   input  shape : {64} output shape :

  0%|          | 0/469 [00:00<?, ?it/s]

epoch[0] : loss=1.683869 accuracy=0.785100


  0%|          | 0/469 [00:00<?, ?it/s]

epoch[1] : loss=1.714527 accuracy=0.754300


...

epoch[5] : loss=1.697749 accuracy=0.761700


  0%|          | 0/469 [00:00<?, ?it/s]

epoch[6] : loss=1.681739 accuracy=0.793000


  0%|          | 0/469 [00:00<?, ?it/s]

epoch[7] : loss=1.681137 accuracy=0.808700


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [22]:
# export verilog
bb.load_networks(data_path, net)

os.makedirs(data_path, exist_ok=True)
os.makedirs(rtl_sim_path, exist_ok=True)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net)

# Simulation用ファイルに上書きコピー
shutil.copyfile(output_velilog_file, sim_velilog_file)

'../../verilog/mnist/tb_mnist_lut4_simple/MnistLutSimple.v'

In [23]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [24]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、6つの入力と接続されており、layer0 の出力ノードは 1024 個あるので、1024x4 の行列が取得できます。

In [25]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(1024, 4)


array([[531, 302, 316, 420],
       [736, 111,  73, 191],
       [529, 203, 400,  20],
       ...,
       [ 96, 287, 745, 140],
       [ 39,  75, 762,  24],
       [730, 179,  33, 337]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
6入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [26]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(1024, 16)


array([[ True,  True,  True, ..., False,  True,  True],
       [False, False, False, ..., False,  True, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [False,  True, False, ...,  True, False,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [False, False,  True, ..., False,  True,  True]])

### 重み行列を覗いてみる

6入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [27]:
W = layer0.W().numpy()
print(W.shape)
W

(1024, 16)


array([[0.49572945, 0.54492617, 0.5558219 , ..., 0.30538175, 0.63873696,
        0.6374619 ],
       [0.5291626 , 0.22972086, 0.52504045, ..., 0.4609878 , 0.5220839 ,
        0.4748494 ],
       [0.6427678 , 0.44930115, 0.3630191 , ..., 0.48375192, 0.39302585,
        0.299717  ],
       ...,
       [0.4619967 , 0.72576404, 0.49766552, ..., 0.68245655, 0.36864707,
        0.5230604 ],
       [0.34179386, 0.5036218 , 0.5010163 , ..., 0.54461145, 0.5709137 ,
        0.6086369 ],
       [0.46385154, 0.4701717 , 0.6976064 , ..., 0.44916087, 0.54175824,
        0.55022866]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [28]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(1024, 16)


array([[-7.3569681e-05, -2.4523251e-05,  1.3952376e-07, ...,
         3.2989828e-06,  1.4579733e-05,  4.8599159e-06],
       [-1.7782324e-05, -7.3976116e-07, -5.9286249e-06, ...,
         1.9758445e-06,  1.3992903e-06,  6.5861605e-07],
       [ 1.3717473e-03,  4.5830509e-04,  6.6864252e-04, ...,
        -1.8283662e-04, -1.3267613e-04, -4.1058367e-05],
       ...,
       [ 1.5365890e-03,  5.9266901e-04, -1.6173377e-03, ...,
         6.5884153e-05, -1.7967775e-04, -5.6912104e-05],
       [-3.7252903e-09, -9.3132257e-10, -9.3132257e-10, ...,
        -4.6566129e-10, -4.6566129e-10, -1.1641532e-10],
       [-4.6566129e-10, -2.9103830e-11,  4.0745363e-10, ...,
        -1.8189894e-11,  5.2750693e-11,  6.3664629e-12]], dtype=float32)